In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
#import more_itertools as mit

In [3]:

FilePathList = "/Users/homefolder/Documents/RapfLab/code/MStest" #sets filepath for initial file/mass list. May not be necessary to keep this way
                                                            #but sets path for list 
FileType="csv" #sets filetype
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
os. chdir(FilePathList) #sets working directory to FilePathList
CurrFile = pd.read_csv("IncompCombinedmasslist.csv") #reads the file masslist.csv from the filepath --> since we only want to read one file was easier to just specify file
#print(CurrFile)
MassNumberNames=[]
#######################################


for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the colum mz_list
    MassNumberNames.append(CurrFile['Species'][i])#appends name associated with mass number
#print(MassNumberNames)

In [22]:
##WEIRD ERROR. SOMETIMES NEED TO RUN TWICE BECAUSE FILEPATH DOES NOT UPDATE??????
####Different way of coding it
FilePath = "/Users/homefolder/Documents/RapfLab/code/MassSpecData/210623MSCHR/MS" #Second file path for where the actual files are located
os. chdir(FilePath) #updates file path to the folder with actual files for processing
AllFiles=pd.DataFrame()
FileType="csv" #what filetype are the files. Will probably be csv
files = sorted(glob.glob("*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in
OutFolder = "Processed" #folder where processed files will be saved
outname = "" #part of file name. Will be used later
##############################
print(files)
ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
ProcessedX=[] #values for X(Thompsons) will be saved here
ProcessedY=[] #values for Y(Counts) will be saved here
PPM=[] #PPM values will be saved here
NameList=[] #Saves names that will be used for file naming later on
CompNames=[]

for entry in range(num_files): #goes through all files
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=1) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
#     CompSplit=name.split("_")
#     for i in range(len(CompSplit)):
#         for f in range(10):
#             if (CompSplit[i][0]) == "0" or (CompSplit[i]) == "MS.csv" or (CompSplit[i][0]) == "1" or (CompSplit[i][0]) == "2" or (CompSplit[i][0]) == "3" or (CompSplit[i][0]) == "4" or (CompSplit[i][0]) == "5" or (CompSplit[i][0]) == "6" or (CompSplit[i][0]) == "7" or (CompSplit[i][0]) == "8" or (CompSplit[i][0]) == "9":
#                 continue
#             else:
#                 CompNames.append(CompSplit[i])
#     print(CompNames)
    NameList.append(str(FullSplit[0]))
    
    
    #gets the third part of the name (0,1,[2]). Adds it to the name list
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns="#Point") #drops point column
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high

        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)

             
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                              # number of values no matter what
                  ProcessedX.append(0)  
                  ProcessedY.append(0) #appends 0 for both 
                  PPM.append(0)
        
        
        

length=(int(len(MassNumbersList))) #length is the lenght of the MassNumbersList
for i in range((num_files)):   
    IndFile = pd.DataFrame(columns = ["Compound Names","mz_list",NameList[i]+"_mass",NameList[i]+"_count"]) #columns mz_list, mass, and count +NameList[i]
    IndFile["Compound Names"]=MassNumberNames
    IndFile["mz_list"]=MassNumbersList
    IndFile[NameList[i]+"_mass"]=ProcessedX[(i*length):((i+1)*length)] 
    IndFile[NameList[i]+"_count"]=ProcessedY[(i*length):((i+1)*length)] #for this and the line before, it reads the values of processed from i*length
                                                                        # to (i+1)*length. For example this will be 0:12 for first then 13:16 for second
    IndFile[NameList[i]+"_PPM"]=PPM[(i*length):((i+1)*length)]
    IndFile.to_csv("/Users/homefolder/Documents/RapfLab/code/MStest/Processed"+"/"+NameList[i]+"_Processed.csv")          #chooses folder
####################


AllFiles[outname+"mz_list"]=MassNumbersList   #Does same as for IndFile but for an All file with all values. Whole function works in similar way
AllFiles[outname+"Compound Names"]=MassNumberNames
for i in range(num_files):
    AllFiles[outname+"_mass"+NameList[i]]=ProcessedX[(i*length):((i+1)*length)]
    AllFiles[outname+"_count"+NameList[i]]=ProcessedY[(i*length):((i+1)*length)]
    AllFiles[outname+"_PPM"+NameList[i]]=PPM[(i*length):((i+1)*length)]
        
AllFiles.to_csv("/Users/homefolder/Documents/RapfLab/code/MStest/Processed"+"/"+outname+"_AllProcessed.csv")

    

['210622_25C_365_OOAPost_MS.csv', '210622_25C_365_PAVacPost_MS.csv', '210622_25C_trPABlkrPost_MS.csv', '210622_25C_trPASurPost_MS.csv', '210622_25C_trPAVacrPost_MS.csv', '210622_25C_trPApre_MS.csv', '210622_365_OOAPost_MS.csv', '210622_365_PAPost_2_MS.csv', '210622_365_PAPost_MS.csv', '210622_trPABlkPost_MS.csv', '210622_trPABlkrPost_2_MS.csv', '210622_trPAPre_2_MS.csv', '210622_trPASurPost_2_MS.csv', '210622_trPASurPost_MS.csv', '210622_trPAVacPost_2_MS.csv', '210622_trPAVacPost_MS.csv', '210622_trPApre_MS.csv', 'MeOHblank1_MS.csv', 'MeOHblank2_MS.csv', 'MeOHblank3_MS.csv', 'MeOHblank4_MS.csv', 'MeOHblank5_MS.csv']
